In [1]:
import numpy as np
import torch 
import torch.nn as nn
from tqdm import tnrange
import torch.optim as optim
import torch.nn.functional as F
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score
import scipy
import time

#from models.sub_cos_glm import Sub_Cos_GLM
#from models.sub_tcn import Sub_TCN
from models.gru import GRU

In [2]:
base_dir = "/media/hdd01/sklee/"
experiment = "clust4-60_noNA"
cell_type = "CA1"
E_neural_file = "Espikes_neural.npz"
I_neural_file = "Ispikes_neural.npz"
clust_mode = "hand"
model_type = "gru"

E_neural = scipy.sparse.load_npz(base_dir+cell_type+"_"+experiment+"/data/"+E_neural_file)
I_neural = scipy.sparse.load_npz(base_dir+cell_type+"_"+experiment+"/data/"+I_neural_file)

if (clust_mode == "hand") or (clust_mode == "whole"):
    C_syn_e = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/handsub10_C_syn_e.npy")
    C_syn_i = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/handsub10_C_syn_i.npy")
    C_syn_e = torch.from_numpy(C_syn_e).float()
    C_syn_i = torch.from_numpy(C_syn_i).float()
elif clust_mode == "rand":
    C_syn_e = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/randsub10_C_syn_e.npy")
    C_syn_i = np.load("/media/hdd01/sklee/"+cell_type+"_"+experiment+"/data/randsub10_C_syn_i.npy")
    C_syn_e = torch.from_numpy(C_syn_e).float()
    C_syn_i = torch.from_numpy(C_syn_i).float()

In [3]:
H_no = 20
sub_no = 10
E_no = 2000
I_no = 200
T_no = 500
device = torch.device("cuda")

In [4]:
if model_type == "gru":
    model = GRU(C_syn_e.to(device), C_syn_i.to(device), H_no, device)
#elif model_type == "tcn":
    #model = Sub_TCN(C_syn_e.to(device), C_syn_i.to(device), T_no, H_no, device)
#elif model_type == "glm":
    #model = Sub_Cos_GLM(C_syn_e.to(device), C_syn_i.to(device), T_no, H_no, device)
    
model.to(device).float()
model.load_state_dict(torch.load(base_dir+cell_type+"_"+experiment+"/"+clust_mode+"/"+model_type+"_s"+str(sub_no)+"_h"+str(H_no)+".pt", map_location='cuda:0'))
model.eval()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

16211


In [5]:
if model_type == "gru":
    test = np.zeros((980,50000))

    for i in tnrange(980):
        part_E_neural = torch.from_numpy(E_neural[(i)*50000:(i+1)*50000].toarray()).to(device).float().unsqueeze(0)
        part_I_neural = torch.from_numpy(I_neural[(i)*50000:(i+1)*50000].toarray()).to(device).float().unsqueeze(0)

        
        part_test, part_sub_out = model(part_E_neural, part_I_neural)
        test[i] = part_test.cpu().detach().numpy().flatten()
        
    np.savez(base_dir+cell_type+"_"+experiment+"/"+clust_mode+"/"+model_type+"_s"+str(sub_no)+"_h"+str(H_no)+"_extra_output.npz",
        test=test)
    



<ipython-input-5-6af643bd40cc>:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(980):


  0%|          | 0/980 [00:00<?, ?it/s]